In [1]:
from datasets import *
from debugging import *
from analytics import *
from augmentations import *
from models import *

import keras_cv
import keras
import tensorflow as tf

2025-08-03 16:46:39.380939: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-03 16:46:39.587579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754235999.656946   22329 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754235999.684867   22329 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754235999.846166   22329 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
from tensorflow.keras.mixed_precision import set_global_policy

set_global_policy('mixed_float16')

In [4]:
vds = composite_ds(
    coin_ds('./datasets/090/annotations.xml', d_img=False, d_bb=False, tsz=(1024, 1024), float_type="float16").repeat(),
    coin_ds('./datasets/190/annotations.xml', d_img=False, d_bb=False, tsz=(1024, 1024), float_type="float16").repeat()
)

sds = coin_ds('./datasets/simple/annotations.xml', d_img=False, d_bb=False, tsz=(1024, 1024), float_type="float16")
tds = composite_ds(
    sds,
    sds.map(apply_directional_blur(45, 15)),
    sds.map(apply_gaussian_blur(21)),
    sds.map(apply_gaussian_blur(11))
)

model = efficientnetv2_retinanet()
model.fit(
    tds.batch(2),
    validation_data=vds.batch(1),
    epochs=100
)


Epoch 1/100


2025-08-03 16:53:28.100833: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng23{k2=7,k12=-1,k13=0,k14=4,k15=0,k17=128,k23=0} for conv %cudnn-conv-bw-input.51 = (f16[2,128,128,256]{3,2,1,0}, u8[0]{0}) custom-call(f16[2,128,128,256]{3,2,1,0} %bitcast.58070, f16[256,3,3,256]{3,2,1,0} %bitcast.57595), window={size=3x3 pad=1_1x1_1}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convBackwardInput", metadata={op_type="Conv2DBackpropInput" op_name="gradient_tape/retina_net_1_1/prediction_head_3_9/conv2d_30_1/convolution/Conv2DBackpropInput" source_file="/mnt/c/Users/n3/Desktop/od-eval/.linuxvnev/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false} is taking a while...
2025-08-03 16:53:28.101507: E exter

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - box_loss: 14.0364 - classification_loss: 11895.0098 - loss: 11909.0449

2025-08-03 16:53:42.814450: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


40/40 ━━━━━━━━━━━━━━━━━━━━ 37s 143ms/step - box_loss: 43.2594 - classification_loss: 35215.6250 - loss: 35258.8828
Epoch 2/100


...

2025-08-03 16:54:03.236103: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 1670439866159970649
2025-08-03 16:54:03.236129: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5678119111121765399
2025-08-03 16:54:03.236164: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4889809804863541806


40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - box_loss: 15.0377 - classification_loss: nan - loss: nan
Epoch 7/100


2025-08-03 16:54:07.256820: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 1670439866159970649
2025-08-03 16:54:07.256846: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 5678119111121765399
2025-08-03 16:54:07.256877: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 4889809804863541806


22/40 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - box_loss: 4.6509 - classification_loss: nan - loss: nan

KeyboardInterrupt: 

## analyse
get key values for optimal performance later on

# training model

In [10]:
from datasets import *
from augmentations import *
from models import *
import tensorflow as tf
import time


sds = coin_ds('./datasets/simple/annotations.xml', d_img=False, d_bb=False, tsz=(1024, 1024), float_type="float16")
tds = composite_ds(
    sds,
    sds.map(apply_directional_blur(45, 15)),
    sds.map(apply_gaussian_blur(21)),
    sds.map(apply_gaussian_blur(11))
)

print("cardinality")
print(tf.data.experimental.cardinality(sds))
print(tf.data.experimental.cardinality(tds))


cardinality
tf.Tensor(20, shape=(), dtype=int64)
tf.Tensor(80, shape=(), dtype=int64)


Epoch 1/100


/mnt/c/Users/n3/Desktop/od-eval/.linuxvnev/lib/python3.12/site-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: {'images': 'images'}
Received: inputs=Tensor(shape=(None, 1024, 1024, 3))
  warnings.warn(msg)


3.png
11.png
3.png
11.png
3.png
11.png
3.png
11.png
4.png
4.png
4.png
4.png


I0000 00:00:1754235885.032669    5642 service.cc:152] XLA service 0xf00b810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754235885.032948    5642 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9


2.png


2025-08-03 16:44:45.407399: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1754235886.946515    5642 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-03 16:44:48.122394: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Constant folding an instruction is taking > 1s:

  %negate.72 = f32[196416,2]{1,0} negate(f32[196416,2]{1,0} %constant.17155), metadata={op_type="Sub" op_name="retina_net_label_encoder_1/sub_31" source_file="/mnt/c/Users/n3/Desktop/od-eval/.linuxvnev/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd li

 1/20 ━━━━━━━━━━━━━━━━━━━━ 10:57 35s/step - box_loss: 0.0000e+00 - classification_loss: 0.0000e+00 - loss: 0.0000e+002.png
2.png
2.png
13.png
 2/20 ━━━━━━━━━━━━━━━━━━━━ 4s 244ms/step - box_loss: 0.0000e+00 - classification_loss: 0.0000e+00 - loss: 0.0000e+00 13.png
13.png
13.png
18.png
 3/20 ━━━━━━━━━━━━━━━━━━━━ 4s 240ms/step - box_loss: 0.0831 - classification_loss: 0.1485 - loss: 0.2315            18.png
 4/20 ━━━━━━━━━━━━━━━━━━━━ 3s 241ms/step - box_loss: 0.1090 - classification_loss: 0.1949 - loss: 0.303918.png
18.png
9.png
9.png
 5/20 ━━━━━━━━━━━━━━━━━━━━ 3s 252ms/step - box_loss: 0.5520 - classification_loss: 1310.6613 - loss: 1311.21349.png
9.png
10.png
10.png
 6/20 ━━━━━━━━━━━━━━━━━━━━ 3s 258ms/step - box_loss: 0.7828 - classification_loss: 2002.2910 - loss: 2003.073710.png
10.png
7.png
7.png
 7/20 ━━━━━━━━━━━━━━━━━━━━ 3s 262ms/step - box_loss: 0.9081 - classification_loss: 2384.8745 - loss: 2385.78277.png
7.png
12.png
12.png
 8/20 ━━━━━━━━━━━━━━━━━━━━ 3s 265ms/step - box_loss:

/mnt/c/Users/n3/Desktop/od-eval/.linuxvnev/lib/python3.12/site-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: {'images': 'images'}
Received: inputs=Tensor(shape=(1024, 1024, 3))
  warnings.warn(msg)


ValueError: Exception encountered when calling RetinaNet.call().

[1mInvalid input shape for input Tensor("data:0", shape=(1024, 1024, 3), dtype=float32). Expected shape (None, None, None, 3), but input has incompatible shape (1024, 1024, 3)[0m

Arguments received by RetinaNet.call():
  • inputs=tf.Tensor(shape=(1024, 1024, 3), dtype=float32)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>

In [35]:

for grid in grid_search({
    'input_size': [(512, 2)],  # h and w, batch size  # 256, 384, 512, 640, 1024
}):
    side, batch_sz = grid['input_size']
    tsz = (side, side)

    with tf.device('/GPU:0'):
        dss = coin_ds('./datasets/synthetic/annotations.xml', d_img=False, d_bb=False)

        ds090 = coin_ds(
            './datasets/090/annotations.xml', 'png', d_img=False, d_bb=False
        ).batch(batch_sz).prefetch(tf.data.AUTOTUNE)

        train_ds = composite_ds(
            dss,
            dss.map(apply_directional_blur(45, 15)),
            # dss.map(apply_gaussian_noise(stddev=0.02)),
            # dss.map(apply_gaussian_noise(stddev=0.02, monochromatic=True)),
            # dss.map(apply_gaussian_blur(21)),
            # dss.map(apply_gaussian_blur(11))
        ).batch(batch_sz).prefetch(tf.data.AUTOTUNE)

        # model = keras_cv.models.RetinaNet(
        #     num_classes=1,
        #     bounding_box_format="xywh",
        #     backbone=keras_cv.models.MobileNetV3Backbone.from_preset('mobilenet_v3_small'),
        #     trainable_backbone=False
        # )




2025-08-01 03:58:22.288819: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.25MiB (rounded to 2359296)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-08-01 03:58:22.290031: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-08-01 03:58:22.290159: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 628, Chunks in use: 628. 157.0KiB allocated for chunks. 157.0KiB in use in bin. 75.3KiB client-requested in use in bin.
2025-08-01 03:58:22.290171: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 391, Chunks in use: 391. 203.2KiB allocated for chunks. 203.2KiB in use in bin. 185.1KiB client-requested in use in bin.
2

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 